In [ ]:
import pandas as pd


In [223]:
df = pd.read_csv('/content/train.csv')

In [224]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [225]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [226]:
df=df.drop(['text','author','id'],axis=1)
#we will be using title columns for our prediction 

In [227]:
df.isnull().sum()# there are some null values in the title columns 


title    558
label      0
dtype: int64

In [228]:
df = df.dropna()#as title is the only column is the what we are using if it contains NaN values we have to drop it.


In [229]:
df.shape

(20242, 2)

In [24]:
import tensorflow as tf

In [25]:
tf.__version__

'2.2.0'

In [27]:
#importing all necessary modules that we will be using to build our LSTM neural network
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [28]:
### Vocabulary size
voc_size=5000

In [230]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [231]:
## Get the Dependent features
y=df['label']

In [232]:
X.shape


(20242, 1)

In [233]:
y.shape

(20242,)

#One hot representation

In [234]:
messages=X.copy()

In [235]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [236]:
messages.reset_index(inplace=True)#as we dropped some rows so to make the dataframe in order



In [237]:
messages=messages.drop(['index'],axis=1)

In [42]:
# importing nltk,stopwords and porterstemmer we are using stemming on the text we have and stopwords will help in removing the stopwords in the text
#re is regular expressions used for identifying only words in the text and ignoring anything else
import nltk
import re
from nltk.corpus import stopwords

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [238]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [239]:
corpus = []
#each row of the dataset is considered here.everything except the alphabets are removed ,stopwords are also being removed here .the text is converted in lowercase letters and stemming is performed
#lemmatisation can also be used here at the end a corpus of sentences is create
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
15243
15244
15245
15246
15247
15248
15249
15250
15251
15252
15253
15254
15255
15256
15257
15258
15259
15260
15261
15262
15263
15264
15265
15266
15267
15268
15269
15270
15271
15272
15273
15274
15275
15276
15277
15278
15279
15280
15281
15282
15283
15284
15285
15286
15287
15288
15289
15290
15291
15292
15293
15294
15295
15296
15297
15298
15299
15300
15301
15302
15303
15304
15305
15306
15307
15308
15309
15310
15311
15312
15313
15314
15315
15316
15317
15318
15319
15320
15321
15322
15323
15324
15325
15326
15327
15328
15329
15330
15331
15332
15333
15334
15335
15336
15337
15338
15339
15340
15341
15342
15343
15344
15345
15346
15347
15348
15349
15350
15351
15352
15353
15354
15355
15356
15357
15358
15359
15360
15361
15362
15363
15364
15365
15366
15367
15368
15369
15370
15371
15372
15373
15374
15375
15376
15377
15378
15379
15380
15381
15382
15383
15384
15385
15386
15387
15388
15389
15390
15391
15392
15393
15394
15395
15396
15397
15398
15399
15400
1

In [240]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [241]:
sent_length=25#specifying a sentence length so that every sentence in the corpus will be of same length
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)#using padding for creating equal length sentences
print(embedded_docs)

[[   0    0    0 ... 4971 3753  893]
 [   0    0    0 ...  814   62 2818]
 [   0    0    0 ... 1903    4 4666]
 ...
 [   0    0    0 ... 3068 3235  453]
 [   0    0    0 ... 4582 1600 1118]
 [   0    0    0 ... 1431   56 3426]]


In [242]:
## Creating model
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout_10 (Dropout)         (None, 25, 40)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 200)               192800    
_________________________________________________________________
dropout_11 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 201       
Total params: 393,001
Trainable params: 393,001
Non-trainable params: 0
_________________________________________________________________
None


In [243]:
len(embedded_docs),y.shape

(20242, (20242,))

In [244]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [245]:
X_final.shape,y_final.shape


((20242, 25), (20242,))

In [246]:
#splitting the data for training and testing the model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.10, random_state=42)

In [247]:
#Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
285/285 [==============================] - 24s 86ms/step - loss: 0.2967 - accuracy: 0.8678 - val_loss: 0.1934 - val_accuracy: 0.9190
Epoch 2/20
285/285 [==============================] - 24s 84ms/step - loss: 0.1373 - accuracy: 0.9474 - val_loss: 0.1888 - val_accuracy: 0.9215
Epoch 3/20
285/285 [==============================] - 24s 84ms/step - loss: 0.0969 - accuracy: 0.9649 - val_loss: 0.2077 - val_accuracy: 0.9230
Epoch 4/20
285/285 [==============================] - 24s 84ms/step - loss: 0.0681 - accuracy: 0.9747 - val_loss: 0.2448 - val_accuracy: 0.9126
Epoch 5/20
285/285 [==============================] - 24s 84ms/step - loss: 0.0502 - accuracy: 0.9818 - val_loss: 0.3084 - val_accuracy: 0.9170
Epoch 6/20
285/285 [==============================] - 24s 84ms/step - loss: 0.0384 - accuracy: 0.9864 - val_loss: 0.3164 - val_accuracy: 0.9151
Epoch 7/20
285/285 [==============================] - 24s 85ms/step - loss: 0.0290 - accuracy: 0.9897 - val_loss: 0.3555 - val_accuracy:

In [248]:
y_pred=model.predict_classes(X_test)

In [249]:
from sklearn.metrics import confusion_matrix

In [250]:
confusion_matrix(y_test,y_pred)

array([[899, 119],
       [ 69, 938]])

In [251]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9071604938271605

In [252]:
#loading test dataset for prediction

df_test = pd.read_csv('/content/test.csv')

In [253]:
df_test["title"].fillna("Fake Fake Fake",inplace = True)#the solution file that can be submitted in kaggle expects it to have 5200 rows so we can't drop rows in the test dataset
df_test = df_test.drop(["text","author","id"],axis =1)#using the title column only as we did in the train dataset



In [254]:
messages1 = df_test.copy()

In [255]:
ps1 = PorterStemmer()

In [256]:
messages1["title"] = messages1["title"].apply(str)

In [258]:
corpus1 = []#creating corpus for the test dataset exactly the same as we created for the training dataset
for i in range(0, len(messages1)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages1['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps1.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus1.append(review)

Streaming output truncated to the last 5000 lines.
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
4

In [259]:
onehot_repr1=[one_hot(words,voc_size)for words in corpus1] #creating one hot representation for the test corpus



In [260]:
sent_length=25#padding for the test dataset

embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length)
print(embedded_docs1)

[[   0    0    0 ... 3068 3235  453]
 [   0    0    0 ... 3463 2876 2228]
 [   0    0    0 ...  439 2734  975]
 ...
 [   0    0    0 ... 3068 3235  453]
 [   0    0    0 ... 4119 4966 3370]
 [   0    0    0 ... 3068 3235  453]]


In [261]:
X_final1=np.array(embedded_docs1)

In [262]:
y_pred_1=model.predict_classes(X_final1)#making predictions for the test dataset


In [263]:
submit_sample=pd.read_csv('/content/submit.csv')


In [265]:
submit_sample.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [266]:
y_pred_1

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [267]:
val=[]
for i in y_pred_1:
    val.append(i[0])

In [268]:
submit_sample['label']=val #inserting our predicted values in the submission file



In [269]:
submit_sample.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [270]:
submit_sample.to_csv("final_test.csv",index = False)#saving the submission file


In [ ]:
# Above model accuracyscore is around 92.5%

In [271]:
submit_sample

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1
...,...,...
5195,25995,0
5196,25996,0
5197,25997,0
5198,25998,0


In [272]:
from tensorflow.keras.layers import Bidirectional# importing for creating a Bidirectional lstm

In [282]:
embedding_vector_features=40# second model creation
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(200)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 25, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 400)               385600    
_________________________________________________________________
dropout_13 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 401       
Total params: 586,001
Trainable params: 586,001
Non-trainable params: 0
_________________________________________________________________
None


In [283]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)#training the second model


Epoch 1/10
285/285 [==============================] - 45s 158ms/step - loss: 0.2659 - accuracy: 0.8891 - val_loss: 0.2308 - val_accuracy: 0.9096
Epoch 2/10
285/285 [==============================] - 49s 173ms/step - loss: 0.1514 - accuracy: 0.9432 - val_loss: 0.2072 - val_accuracy: 0.9101
Epoch 3/10
285/285 [==============================] - 46s 161ms/step - loss: 0.1055 - accuracy: 0.9612 - val_loss: 0.2112 - val_accuracy: 0.9185
Epoch 4/10
285/285 [==============================] - 46s 160ms/step - loss: 0.0667 - accuracy: 0.9769 - val_loss: 0.2673 - val_accuracy: 0.9195
Epoch 5/10
285/285 [==============================] - 46s 161ms/step - loss: 0.0452 - accuracy: 0.9842 - val_loss: 0.3474 - val_accuracy: 0.9146
Epoch 6/10
285/285 [==============================] - 46s 161ms/step - loss: 0.0308 - accuracy: 0.9897 - val_loss: 0.3570 - val_accuracy: 0.9151
Epoch 7/10
285/285 [==============================] - 46s 161ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.4328 - val_ac

In [284]:
y_pred_11=model1.predict_classes(X_final1)#making predictions for the test dataset using the second dataset

In [285]:
val1=[]
for i in y_pred_11:
    val1.append(i[0])

In [286]:
submit_sample['label']=val1 #inserting our predicted values in the submission file


In [287]:
submit_sample.to_csv("final_test.csv",index = False)#saving the submission file 
# this model accuracy score is around 93.3%

In [288]:
submit_sample

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1
...,...,...
5195,25995,0
5196,25996,0
5197,25997,0
5198,25998,1
